<a href="https://www.kaggle.com/code/andrey36912/notebookac402befd2?scriptVersionId=192771530" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [10]:
import numpy as np
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, ConcatDataset
import torch
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from PIL import Image

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"


In [12]:
data_dir = '/kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_0'


dataset = datasets.ImageFolder(root=data_dir)

In [13]:
data_dir = '/kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_1'
dataset1 = datasets.ImageFolder(root=data_dir)

In [14]:
data_dir = '/kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_2'
dataset2 = datasets.ImageFolder(root=data_dir)

In [15]:
classes = dataset_0.classes
num_classes = len(classes)

NameError: name 'dataset_0' is not defined

In [ ]:
dataset = ConcatDataset([dataset_0, dataset_1, dataset_2])
print(f"Количество классов: {num_classes}")
print(f"Классы: {classes}")

In [ ]:
model = FractureClassifier(num_classes).to(device)
print(model)

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])


In [ ]:
len(train_data), len(val_data)

In [ ]:
train_ratio = 0.8
train_size = int(train_ratio * len(dataset))
val_size = len(dataset) - train_size
train_data, val_data = random_split(dataset, [train_size, val_size])
train_data.dataset.transform = train_transform
val_data.dataset.transform = test_transform

In [ ]:
len(train_data), len(val_data)

In [ ]:
batch_size = 32
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=False, batch_size=batch_size)

In [ ]:
class ImageDataset(torch.utils.data.Dataset):
   
    def __init__(self, root_dir, transform=None):
       
        self.root_dir = root_dir
        self.image_paths, self.labels = self.get_paths_and_labels()
        self.transform = transform

    def get_paths_and_labels(self):
        image_paths = []
        labels = []
        for label in os.listdir(self.root_dir):
            class_dir = os.path.join(self.root_dir, label)
            if os.path.isdir(class_dir):
                for img_name in os.listdir(class_dir):
                    img_path = os.path.join(class_dir, img_name)
                    image_paths.append(img_path)
                    labels.append(label)
        return image_paths, labels

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        label_name = self.labels[idx]
        label = 0 if label_name == 'all' else 1  
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
os.listdir(data_dir)

In [ ]:
custom_train_data = ImageDataset(data_dir, transform=transform)

In [ ]:

batch_size = 32
train_loader = DataLoader(custom_train_data, shuffle=True, batch_size=batch_size)

In [ ]:
import matplotlib.pyplot as plt

for i in range(3):  
    img, label = custom_train_data[i]

    img = img.numpy().transpose((1, 2, 0))  

    plt.imshow(img)
    plt.title(f"Image: {i+1}, Label: {label}")
    plt.show()

In [ ]:
class FractureClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3)
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.conv5 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.conv6 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.flatten = nn.Flatten()
        
        self.linear1 = nn.Linear(32*6*6, 256)
        self.linear2 = nn.Linear(256, num_classes)
        
    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(out)
        out = self.conv2(out)
        out = F.relu(out)
        out = self.pool1(out)
        out = self.conv3(out)
        out = F.relu(out)
        out = self.pool2(out)
        out = self.conv4(out)
        out = F.relu(out)
        out = self.pool3(out)
        out = self.conv5(out)
        out = F.relu(out)
        out = self.pool4(out)
        out = self.conv6(out)
        out = F.relu(out)
        out = self.pool5(out)
        out = self.flatten(out)
        out = self.linear1(out)
        out = F.relu(out)
        out = self.linear2(out)
        return out
    
    def predict(self, X, device='cpu'):
        X = torch.FloatTensor(np.array(X)).to(device)
        with torch.no_grad():
            y_pred = self.forward(X)
        return y_pred.cpu().numpy()
model = FractureClassifier(num_classes).to(device)
print(model)
    

In [ ]:
!pip install -q torchsummary

In [ ]:
from torchsummary import summary

summary(model, input_size=(3, 256, 256))

In [ ]:
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score

history = train(model, optimizer, loss_fn, train_loader, val_loader,
                epochs=10,
                metrics=[accuracy_score],
                device=device,
                task='multiclass')

In [ ]:
import matplotlib.pyplot as plt

def plot_metric(history, name):
    plt.title(f"Model results with {name}")
    plt.plot(history[name], label='train')
    plt.plot(history['val_'+name], label='val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    plt.legend()


plot_metric(history, 'loss')

In [ ]:
import numpy as np
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, ConcatDataset
import torch
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"

# Определение путей к данным
data_dir_0 = '/kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_0'
data_dir_1 = '/kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_1'
data_dir_2 = '/kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_2'

# Применение трансформаций
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
])

# Загрузка данных с применением трансформаций
dataset_0 = datasets.ImageFolder(root=data_dir_0, transform=transform)
dataset_1 = datasets.ImageFolder(root=data_dir_1, transform=transform)
dataset_2 = datasets.ImageFolder(root=data_dir_2, transform=transform)

# Извлечение классов
classes = dataset_0.classes
num_classes = len(classes)

# Объединение датасетов
dataset = ConcatDataset([dataset_0, dataset_1, dataset_2])

# Разделение данных на обучающую и валидационную выборки
train_ratio = 0.8
train_size = int(train_ratio * len(dataset))
val_size = len(dataset) - train_size
train_data, val_data = random_split(dataset, [train_size, val_size])

# Загрузчики данных
batch_size = 32
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=False, batch_size=batch_size)

# Определение модели
class FractureClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3)
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.conv5 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.conv6 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.flatten = nn.Flatten()
        
        self.linear1 = nn.Linear(32*6*6, 256)
        self.linear2 = nn.Linear(256, num_classes)
        
    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(out)
        out = self.conv2(out)
        out = F.relu(out)
        out = self.pool1(out)
        out = self.conv3(out)
        out = F.relu(out)
        out = self.pool2(out)
        out = self.conv4(out)
        out = F.relu(out)
        out = self.pool3(out)
        out = self.conv5(out)
        out = F.relu(out)
        out = self.pool4(out)
        out = self.conv6(out)
        out = F.relu(out)
        out = self.pool5(out)
        out = self.flatten(out)
        out = self.linear1(out)
        out = F.relu(out)
        out = self.linear2(out)
        return out
    
    def predict(self, X, device='cpu'):
        X = torch.FloatTensor(np.array(X)).to(device)
        with torch.no_grad():
            y_pred = self.forward(X)
        return y_pred.cpu().numpy()

model = FractureClassifier(num_classes).to(device)
print(model)

# Обучение модели
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Обучение
from sklearn.metrics import accuracy_score

history = train(model, optimizer, loss_fn, train_loader, val_loader,
                epochs=10,
                metrics=[accuracy_score],
                device=device,
                task='multiclass')
